<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
import numpy

In [18]:
churn = pd.read_csv('/Users/mattmastin/Desktop/churn.csv')
churn.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
churn.describe(exclude='number')

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,6531,2
top,1840-BIUOG,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,20.2,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,11,5174


In [8]:
pd.set_option('display.max_columns', None)

In [9]:
churn.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [11]:
churn.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [21]:
# map = {'Yes': 1, 'No': 0}
churn = churn.replace({'Yes': 1, 'No': 0})

In [23]:
churn = churn.replace({'Male': 1, 'Female': 0})

In [26]:
churn.head(30)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,1,0,1,0,No phone service,DSL,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,5575-GNVDE,1,0,0,0,34,1,0,DSL,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.5,0
2,3668-QPYBK,1,0,0,0,2,1,0,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1
3,7795-CFOCW,1,0,0,0,45,0,No phone service,DSL,1,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,0,0,0,0,2,1,0,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1
5,9305-CDSKC,0,0,0,0,8,1,1,Fiber optic,0,0,1,0,1,1,Month-to-month,1,Electronic check,99.65,820.5,1
6,1452-KIOVK,1,0,0,1,22,1,1,Fiber optic,0,1,0,0,1,0,Month-to-month,1,Credit card (automatic),89.10,1949.4,0
7,6713-OKOMC,0,0,0,0,10,0,No phone service,DSL,1,0,0,0,0,0,Month-to-month,0,Mailed check,29.75,301.9,0
8,7892-POOKP,0,0,1,0,28,1,1,Fiber optic,0,0,1,1,1,1,Month-to-month,1,Electronic check,104.80,3046.05,1
9,6388-TABGU,1,0,0,1,62,1,0,DSL,1,1,0,0,0,0,One year,0,Bank transfer (automatic),56.15,3487.95,0


In [28]:
churn.describe(exclude='number')

,customerID,InternetService,Contract,PaymentMethod,TotalCharges
count,7043,7043,7043,7043,7043
unique,7043,3,3,4,6531
top,1840-BIUOG,Fiber optic,Month-to-month,Electronic check,20.2
freq,1,3096,3875,2365,11


In [27]:
churn = churn.replace({'No phone service': 0})
churn = churn.replace({'No internet service': 0})

In [29]:
churn.dtypes

customerID           object
gender                int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
tenure                int64
PhoneService          int64
MultipleLines         int64
InternetService      object
OnlineSecurity        int64
OnlineBackup          int64
DeviceProtection      int64
TechSupport           int64
StreamingTV           int64
StreamingMovies       int64
Contract             object
PaperlessBilling      int64
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                 int64
dtype: object

In [31]:
# import category_encoders as ce

In [33]:
churn2 = churn.copy()

In [34]:
churn = churn.drop(columns=['customerID', 'InternetService', 'Contract', 'PaymentMethod', 'TotalCharges'])

In [35]:
churn.shape

(7043, 16)

In [36]:
churn.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,Churn
0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,29.85,0
1,1,0,0,0,34,1,0,1,0,1,0,0,0,0,56.95,0
2,1,0,0,0,2,1,0,1,1,0,0,0,0,1,53.85,1
3,1,0,0,0,45,0,0,1,0,1,1,0,0,0,42.30,0
4,0,0,0,0,2,1,0,0,0,0,0,0,0,1,70.70,1


In [39]:
X = churn.drop(columns=['Churn'])
y = churn['Churn']

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [41]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5282, 15), (1761, 15), (5282,), (1761,))

In [60]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(15,)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [61]:
model.fit(X_train, y_train, epochs=10, batch_size=100)

Epoch 1/10
5282/5282 [==============================] - 5s 930us/step - loss: 0.5209 - acc: 0.7516
Epoch 2/10
5282/5282 [==============================] - 0s 76us/step - loss: 0.4879 - acc: 0.7760
Epoch 3/10
5282/5282 [==============================] - 0s 42us/step - loss: 0.4887 - acc: 0.7787
Epoch 4/10
5282/5282 [==============================] - 0s 44us/step - loss: 0.4782 - acc: 0.7796
Epoch 5/10
5282/5282 [==============================] - 0s 55us/step - loss: 0.4654 - acc: 0.7893
Epoch 6/10
5282/5282 [==============================] - 0s 41us/step - loss: 0.4636 - acc: 0.7844
Epoch 7/10
5282/5282 [==============================] - 0s 41us/step - loss: 0.4656 - acc: 0.7878
Epoch 8/10
5282/5282 [==============================] - 0s 57us/step - loss: 0.4568 - acc: 0.7876
Epoch 9/10
5282/5282 [==============================] - 0s 43us/step - loss: 0.4531 - acc: 0.7900
Epoch 10/10
5282/5282 [==============================] - 0s 46us/step - loss: 0.4538 - acc: 0.7880


In [51]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 32)                512       
_________________________________________________________________
dense_11 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 33        
Total params: 1,601
Trainable params: 1,601
Non-trainable params: 0
_________________________________________________________________


In [53]:
results = model.evaluate(X_test, y_test)
results

1761/1761 [==============================] - 0s 66us/step


[0.44078138262490396, 0.8006814310051107]

In [58]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

param_grid = {'batch_size': [20, 50, 100],
              'epochs': [10, 20, 100, 200],
             }

def create_model():
    model = Sequential()
    model.add(Dense(32, input_dim=15, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f'means: {mean}, stdev:, {stdev} with: {param}')

Best: 0.8013630639536888 using {'batch_size': 50, 'epochs': 200}
means: 0.7962515936746831, stdev:, 0.0035593786393482633 with: {'batch_size': 20, 'epochs': 10}
means: 0.7942638056328668, stdev:, 0.0024193725804488786 with: {'batch_size': 20, 'epochs': 20}
means: 0.7989493119992537, stdev:, 0.0021329788075618184 with: {'batch_size': 20, 'epochs': 100}
means: 0.7936958715554481, stdev:, 0.0013672618995374448 with: {'batch_size': 20, 'epochs': 200}
means: 0.7962515910173129, stdev:, 0.002849315768099459 with: {'batch_size': 50, 'epochs': 10}
means: 0.7988073197630476, stdev:, 0.003193262153280229 with: {'batch_size': 50, 'epochs': 20}
means: 0.7971035122245139, stdev:, 0.0008714457987044348 with: {'batch_size': 50, 'epochs': 100}
means: 0.8013630639536888, stdev:, 0.003737138905612601 with: {'batch_size': 50, 'epochs': 200}
means: 0.7949737235596958, stdev:, 0.0031358512780107835 with: {'batch_size': 100, 'epochs': 10}
means: 0.7938378677692465, stdev:, 0.004860872287126406 with: {'batch

In [62]:
results = model.evaluate(X_test, y_test)
results

1761/1761 [==============================] - 2s 1ms/step


[0.42791720280452317, 0.7904599659284497]

In [64]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler

param_grid = {'batch_size': [20, 100, 200],
              'epochs': [10, 50, 200],
             }

scaler = StandardScaler()
X = scaler.fit_transform(X)

def create_model():
    model = Sequential()
    model.add(Dense(32, input_dim=15, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f'mean: {mean}, stdev:, {stdev} with: {param}')

//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.7971034976597555 using {'batch_size': 20, 'epochs': 10}
mean: 0.7971034976597555, stdev:, 0.0034339789070801444 with: {'batch_size': 20, 'epochs': 10}
mean: 0.7816271435870851, stdev:, 0.005568892880628426 with: {'batch_size': 20, 'epochs': 50}
mean: 0.7599034581386821, stdev:, 0.0038992183198699147 with: {'batch_size': 20, 'epochs': 200}
mean: 0.793553899181815, stdev:, 0.0037266302980479065 with: {'batch_size': 100, 'epochs': 10}
mean: 0.7911401516450461, stdev:, 0.0016060427201309527 with: {'batch_size': 100, 'epochs': 50}
mean: 0.7658668165093167, stdev:, 0.004388917303895073 with: {'batch_size': 100, 'epochs': 200}
mean: 0.7911401273648069, stdev:, 0.003304291109671092 with: {'batch_size': 200, 'epochs': 10}
mean: 0.796535585213669, stdev:, 0.0020171805729483116 with: {'batch_size': 200, 'epochs': 50}
mean: 0.7790714223479981, stdev:, 0.006690230373894218 with: {'batch_size': 200, 'epochs': 200}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?